In [1]:
import logging
LOG_FILENAME = '/core/logs/backtest.log'
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',filename=LOG_FILENAME)
from cdcqr.backtest.vbt.runsignal import runsignal
from cdcqr.common.utils import parallel_jobs
import numpy as np
from cdcqr.common.utils import LOCAL_DATA_DIR, dict_hash
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from cdcqr.common.utils import LOCAL_DATA_DIR
import json
from cdcqr.data.glassnode.features.feature_loaders import produce_feature_BTCSSR

In [2]:
pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
signalid = 'long-BTCSSR_pctrank'
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))
short=False
asset = 'BTC'
side = 'long'
feature = 'BTCSSR'

In [3]:
feature_data_fname = produce_feature_BTCSSR()
feature_data_dir = os.path.join(LOCAL_DATA_DIR, 'feature_data', '{}.pickle'.format(feature_data_fname))
logging.info('loading feature data {}'.format(feature_data_dir))
df4a = pd.read_pickle(feature_data_dir)

backtest_info = {}
backtest_info['signalid'] = signalid
backtest_info['side'] = 'long'
backtest_info_hashed = dict_hash(backtest_info)
backtest_job_info = '{}_{}'.format(json.dumps(backtest_info), backtest_info_hashed)
job_start_log_info = 'Starting backtest job {}'.format(backtest_job_info)
logging.info(job_start_log_info)

backtest_info = {}
backtest_info['signalid'] = signalid
backtest_info['side'] = 'long'
backtest_info_hashed = dict_hash(backtest_info)

qtl_params = [0.995, 0.99, 0.975, 0.96]
lbw_params = [1000, 2000, 3000, 4000]

tp_params = [3, 5, 7, 10]
sl_params = [1, 2, 3 , 5 , 7, 10]
n1_params = [1, 10, 100]
n2_params = [101, 200, 500, 1000, 2000]

loading market/price_usd_close asset=BTC, resolution=10m
loading market/price_usd_close asset=USDT, resolution=10m
loading market/price_usd_close asset=USDC, resolution=10m
loading market/price_usd_close asset=BUSD, resolution=10m
loading distribution/balance_exchanges asset=BTC, resolution=10m
loading distribution/balance_exchanges asset=USDT, resolution=10m
loading distribution/balance_exchanges asset=USDC, resolution=10m
loading distribution/balance_exchanges asset=BUSD, resolution=10m
saved df to C:\core\tmp\feature_data\gn_BTCSSR.pickle
'produce_feature_BTCSSR'  242.04 sec


In [5]:
df4a

,stable_bal_chg,stable_bal_chg_lag1,BTCSSR,BTCSSR_chg,BTCSSR_chg_lag1,ret,r+5,r+10,r+100
t,,,,,,,,,
2020-01-02 09:20:00,5.898457e+04,-6.781299e+05,25.645165,0.007058,0.012455,0.000345,0.001701,-0.002180,0.006508
2020-01-02 09:30:00,-1.676253e+05,5.898457e+04,25.639231,-0.005934,0.007058,-0.000430,0.003049,-0.005359,0.007669
2020-01-02 09:40:00,1.613140e+06,-1.676253e+05,25.590710,-0.048521,-0.005934,0.000012,0.003838,-0.002824,0.009409
2020-01-02 09:50:00,-1.403237e+06,1.613140e+06,25.649857,0.059146,-0.048521,0.000650,0.002812,-0.001200,0.008430
2020-01-02 10:00:00,1.252565e+06,-1.403237e+06,25.621702,-0.028155,0.059146,0.000462,0.001069,-0.000876,0.009844
...,...,...,...,...,...,...,...,...,...
2022-02-28 07:20:00,3.008923e+06,-6.999979e+06,3.428662,0.012499,0.000941,0.004051,0.007992,0.000372,-0.010159
2022-02-28 07:30:00,2.176862e+06,3.008923e+06,3.438078,0.009416,0.012499,0.002879,0.003450,-0.003256,-0.015097
2022-02-28 07:40:00,-2.665225e+06,2.176862e+06,3.447407,0.009329,0.009416,0.002718,0.001316,0.000120,-0.011459
